In [1]:
from navigate import get_data, open_dat_file
from utils.plotting import figure_compare
from filter import line_filter
import os.path as op
import numpy as np
import mne
import matplotlib

# %% Set up logging
mne.set_log_file("output.log",
                "%(levelname)s: %(message)s - %(asctime)s",
                overwrite=True)
mne.set_log_level("INFO")

In [2]:
# %% Load the data
HOME = op.expanduser("~")
LAB_root = op.join(HOME, "Box", "CoganLab")
layout, raw, D_dat_raw, D_dat_filt = get_data(29, "SentenceRep", lab_root=LAB_root)
raw_dat = open_dat_file(D_dat_raw, raw.copy().ch_names)
dat = open_dat_file(D_dat_filt, raw.copy().ch_names)

C:\Users\ae166\git\IEEG_Pipelines\PreProcess\navigate.py:82: RuntimeWarning: The unit for channel(s) Trigger has changed from V to NA.
  new_raw = read_raw_bids(bids_path=BIDS_path)
C:\Users\ae166\git\IEEG_Pipelines\PreProcess\navigate.py:82: RuntimeWarning: participants.tsv file not found for C:\Users\ae166\Box\CoganLab\BIDS-1.0_SentenceRep\BIDS\sub-D0029\ieeg\sub-D0029_task-SentenceRep_acq-01_run-01_ieeg.edf
  new_raw = read_raw_bids(bids_path=BIDS_path)
C:\Users\ae166\git\IEEG_Pipelines\PreProcess\navigate.py:82: RuntimeWarning: The unit for channel(s) Trigger has changed from V to NA.
  new_raw = read_raw_bids(bids_path=BIDS_path)
C:\Users\ae166\git\IEEG_Pipelines\PreProcess\navigate.py:82: RuntimeWarning: participants.tsv file not found for C:\Users\ae166\Box\CoganLab\BIDS-1.0_SentenceRep\BIDS\sub-D0029\ieeg\sub-D0029_task-SentenceRep_acq-01_run-02_ieeg.edf
  new_raw = read_raw_bids(bids_path=BIDS_path)
C:\Users\ae166\git\IEEG_Pipelines\PreProcess\navigate.py:82: RuntimeWarning: T

In [1]:
# %% Filter the data
# raw.drop_channels(raw.ch_names[10:158])
# raw_dat.drop_channels(raw_dat.ch_names[10:158])
# dat.drop_channels(dat.ch_names[10:158])
# filt = mne.filter.notch_filter(raw.get_data(), raw.info['sfreq'], [60], notch_widths=20, method='spectrum_fit')
filt = line_filter(raw, mt_bandwidth=10.0, n_jobs=-1,
                    filter_length='700ms', verbose=10,
                    freqs=[60], notch_widths=20)
filt2 = line_filter(filt, mt_bandwidth=10.0, n_jobs=-1,
                filter_length='20s', verbose=10,
                freqs=[120, 180, 240], notch_widths=20)


NameError: name 'line_filter' is not defined

In [ ]:
# %% Plot the data
data = [raw, filt, filt2, raw_dat, dat]
figure_compare(data, ["BIDS Un", "BIDS 700ms ", "BIDS 20s+700ms ", "Un",
                    ""], avg=True, verbose=10, proj=True, fmax=250)
figure_compare(data, ["BIDS Un", "BIDS 700ms ", "BIDS 20s+700ms ", "Un",
                    ""], avg=False, verbose=10, proj=True, fmax=250)

In [4]:
# load the data
filt = mne.io.read_raw_fif(layout.root + "/derivatives/sub-D00" + str(29) + "_" + "SentenceRep" + "_filt_ieeg.fif")


In [40]:
freqs = np.arange(10, 200., 2.)
events, event_id = mne.events_from_annotations(filt)
auds = mne.Epochs(filt, events, event_id, tmin=-1, tmax=1.5, baseline=None)['Audio']
power, itc = mne.time_frequency.tfr_multitaper(auds, freqs, time_bandwidth=5.0, n_cycles=freqs / 2, n_jobs=6)
# power[0].plot()

[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  12 tasks      | elapsed:  2.0min
[Parallel(n_jobs=6)]: Done  66 tasks      | elapsed: 10.6min
[Parallel(n_jobs=6)]: Done 140 out of 140 | elapsed: 23.9min finished


In [48]:
# # import matplotlib
# # %matplotlib
# 57 = lpmf9
# power.plot([57], baseline=(None,-.5),mode='mean', vmax=1e-10, vmin=-1e-10)
auds.plot()

<MNEBrowseFigure size 800x800 with 4 Axes>